In [69]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from PIL import Image
from src.benchmarkers import *
from src.benchmarkersV2 import *
from tqdm import tqdm
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from io import BytesIO

np.random.seed(0)

In [50]:
table = pq.read_table(r"C:\Users\Cristiano Lavoro\Downloads\ds_10.parquet")

In [8]:
import pyarrow as pa
import json

# Define the schema with a list of struct fields
#schema = pa.schema([
#    pa.field('id', pa.int32()),
#    pa.field('name', pa.string()),
#    pa.field('addresses', pa.list_(
#        pa.struct([
#            ('street', pa.list_(pa.struct([("prova",pa.string())]))),
#            ('city', pa.string()),
#            ('zipcode', pa.int32())
#        ])
#    ))
#])
schema = table.schema

# Convert pyarrow schema to a JSON serializable format
def schema_to_dict(schema):
    schema_dict = {}
    for field in schema:
        schema_dict[field.name] = field_to_dict(field)
    return schema_dict

def field_to_dict(field):
    if pa.types.is_list(field.type):
        # For list types, extract the value field type (should be a struct in your case)
        return {
            "type": "list",
            "value_type": field_to_dict(field.type.value_field)
        }
    elif pa.types.is_struct(field.type):
        # For struct types, extract the fields
        return {
            "type": "struct",
            "fields": {f.name: field_to_dict(f) for f in field.type}
        }
    else:
        # For primitive types (int, string, etc.)
        return {"type": str(field.type)}

# Convert to JSON
schema_json = json.dumps(schema_to_dict(schema), indent=2)
schema_dict = schema_to_dict(schema)
print(type(schema_dict))
schema_dict

<class 'dict'>


{'image_feature': {'type': 'list',
  'value_type': {'type': 'struct',
   'fields': {'boundingbox_feature': {'type': 'list',
     'value_type': {'type': 'struct',
      'fields': {'bbox': {'type': 'list', 'value_type': {'type': 'double'}},
       'image_1_feature': {'type': 'list',
        'value_type': {'type': 'struct',
         'fields': {'image': {'type': 'binary'},
          'shape': {'type': 'list', 'value_type': {'type': 'int64'}}}}},
       'label_feature': {'type': 'list',
        'value_type': {'type': 'struct',
         'fields': {'label': {'type': 'int64'}}}}}}},
    'image': {'type': 'binary'},
    'shape': {'type': 'list', 'value_type': {'type': 'int64'}},
    'text_1_feature': {'type': 'list',
     'value_type': {'type': 'struct',
      'fields': {'text': {'type': 'string'}}}}}}},
 'text_feature': {'type': 'list',
  'value_type': {'type': 'struct', 'fields': {'text': {'type': 'string'}}}}}

In [41]:
import pyarrow as pa
import json

# A mapping from JSON string types to pyarrow types
TYPE_MAPPING = {
    'int64': pa.int64,
    'int32': pa.int32,
    'float64': pa.float64,  # This is the correct PyArrow type for 'double'
    'double': pa.float64,   # If 'double' is used in the JSON, map it to 'float64'
    'string': pa.string,
    'binary': pa.binary
}

# Convert JSON back to PyArrow Schema
def dict_to_field(name, field_dict):
    field_type = field_dict['type']
    
    if field_type == 'list':
        value_type_dict = field_dict['value_type']
        
        # Check if the list contains a struct or a primitive type
        if 'fields' in value_type_dict:
            # List of struct
            value_type = pa.struct([
                dict_to_field(f_name, f_dict)
                for f_name, f_dict in value_type_dict['fields'].items()
            ])
        else:
            # List of primitive types
            value_type = TYPE_MAPPING[value_type_dict['type']]()

        return pa.field(name, pa.list_(value_type))

    elif field_type == 'struct':
        return pa.field(name, pa.struct([
            dict_to_field(f_name, f_dict)
            for f_name, f_dict in field_dict['fields'].items()
        ]))
    
    else:
        # For primitive types (int, string, binary, etc.)
        return pa.field(name, TYPE_MAPPING[field_type]())

# Convert JSON back to PyArrow Schema
def dict_to_schema(schema_dict):
    fields = [dict_to_field(name, field_dict) for name, field_dict in schema_dict.items()]
    return pa.schema(fields)

# Example: JSON string back to schema
schema_dict = {
  "image_feature": {
    "type": "list",
    "value_type": {
      "type": "struct",
      "fields": {
        "boundingbox_feature": {
          "type": "list",
          "value_type": {
            "type": "struct",
            "fields": {
              "bbox": {
                "type": "list",
                "value_type": {
                  "type": "double"
                }
              },
              "area":{
                "type":"double"  
              },
              "image_1_feature": {
                "type": "list",
                "value_type": {
                  "type": "struct",
                  "fields": {
                    "image": {
                      "type": "binary"
                    },
                    "shape": {
                      "type": "list",
                      "value_type": {
                        "type": "int64"
                      }
                    }
                  }
                }
              },
              "label_feature": {
                "type": "list",
                "value_type": {
                  "type": "struct",
                  "fields": {
                    "label": {
                      "type": "int64"
                    }
                  }
                }
              }
            }
          }
        },
        "image": {
          "type": "binary"
        },
        "shape": {
          "type": "list",
          "value_type": {
            "type": "int64"
          }
        },
        "text_1_feature": {
          "type": "list",
          "value_type": {
            "type": "struct",
            "fields": {
              "text": {
                "type": "string"
              }
            }
          }
        }
      }
    }
  },
  "text_feature": {
    "type": "list",
    "value_type": {
      "type": "struct",
      "fields": {
        "text": {
          "type": "string"
        }
      }
    }
  }
}

# Reconstruct schema
reconstructed_schema = dict_to_schema(schema_dict)
print(reconstructed_schema)

image_feature: list<item: struct<boundingbox_feature: list<item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>, image: binary, shape: list<item: int64>, text_1_feature: list<item: struct<text: string>>>>
  child 0, item: struct<boundingbox_feature: list<item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>, image: binary, shape: list<item: int64>, text_1_feature: list<item: struct<text: string>>>
      child 0, boundingbox_feature: list<item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>
          child 0, item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, 

In [53]:
def create_empty_table(schema, length):
    # Create an empty column for each field in the schema
    columns = []
    for field in schema:
        # Create a column of `None` (null) values for each field
        columns.append(pa.array([None] * length, type=field.type))
    
    # Create and return the table
    table = pa.Table.from_arrays(columns, schema=schema)
    return table

In [54]:
new_table = create_empty_table(reconstructed_schema,10)

In [37]:
new_table = new_table.set_column(0,"image_feature",table.column("image_feature"))

In [61]:
import pyarrow as pa
from typing import List, Union, Callable

def modify_nested_field(table: pa.Table, column_name: str, field_path: List[str], modify_func: Callable) -> pa.Table:
    def recursive_modify(data: Union[pa.Array, pa.ChunkedArray], path: List[str]) -> Union[pa.Array, pa.ChunkedArray]:
        if len(path) == 0:
            return modify_func(data)
        
        current_field = path[0]
        remaining_path = path[1:]
        
        if isinstance(data, pa.ChunkedArray):
            modified_chunks = [recursive_modify(chunk, path) for chunk in data.chunks]
            return pa.chunked_array(modified_chunks)
        
        if pa.types.is_list(data.type):
            modified_values = recursive_modify(data.values, path)
            return pa.ListArray.from_arrays(data.offsets, modified_values)
        
        if pa.types.is_struct(data.type):
            modified_fields = []
            for field in data.type:
                if field.name == current_field:
                    modified_field = recursive_modify(data.field(field.name), remaining_path)
                else:
                    modified_field = data.field(field.name)
                modified_fields.append(pa.field(field.name, modified_field.type))
            return pa.StructArray.from_arrays(
                [data.field(field.name) if field.name != current_field else modified_field for field in data.type],
                fields=modified_fields
            )
        
        raise ValueError(f"Unexpected data type: {data.type}")

    column = table[column_name]
    modified_column = recursive_modify(column, field_path)
    return table.set_column(table.column_names.index(column_name), column_name, modified_column)

# Example usage
if __name__ == "__main__":
    # Create a sample table with a list of structs
    nested_type = pa.list_(pa.struct([
        ('name', pa.string()),
        ('details', pa.list_(pa.struct([
            ('id', pa.int32()),
            ('value', pa.float64())
        ])))
    ]))
    
    data = [
        [
            {'name': 'A', 'details': [{'id': 1, 'value': 1.1}, {'id': 2, 'value': 1.2}]},
            {'name': 'B', 'details': [{'id': 3, 'value': 2.1}, {'id': 4, 'value': 2.2}]}
        ],
        [
            {'name': 'C', 'details': [{'id': 5, 'value': 3.1}, {'id': 6, 'value': 3.2}]},
            {'name': 'D', 'details': [{'id': 7, 'value': 4.1}, {'id': 8, 'value': 4.2}]}
        ]
    ]
    
    # Create the PyArrow Array directly
    nested_array = pa.array(data, type=nested_type)
    
    # Create the table from the array
    table = pa.Table.from_arrays([nested_array], names=['nested'])
    
    # Define a modification function
    def double_value(array):
        if isinstance(array, pa.ChunkedArray):
            return pa.chunked_array([chunk.cast(pa.float64()) * 2 for chunk in array.chunks])
        return array
    
    # Modify the 'value' field in the nested structure
    modified_table = modify_nested_field(table, 'nested', ['details', 'value'], double_value)
    
    print("Original table:")
    print(table.to_pandas())
    print("\nModified table:")
    print(modified_table.to_pandas())

Original table:
                                              nested
0  [{'name': 'A', 'details': [{'id': 1, 'value': ...
1  [{'name': 'C', 'details': [{'id': 5, 'value': ...

Modified table:
                                              nested
0  [{'name': 'A', 'details': [{'id': 1, 'value': ...
1  [{'name': 'C', 'details': [{'id': 5, 'value': ...


In [66]:
modified_table = modify_nested_field(new_table, 'image_feature', ['boundingbox_feature', 'bbox'], double_value)

ArrowInvalid: Struct child array #0 does not match type field: list<item: struct<label: int64>> vs list<item: double>

In [65]:
new_table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("area")

[]

In [63]:
new_table.schema

image_feature: list<item: struct<boundingbox_feature: list<item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>, image: binary, shape: list<item: int64>, text_1_feature: list<item: struct<text: string>>>>
  child 0, item: struct<boundingbox_feature: list<item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>, image: binary, shape: list<item: int64>, text_1_feature: list<item: struct<text: string>>>
      child 0, boundingbox_feature: list<item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>
          child 0, item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, 